In [ ]:
# this codes transcribes the coding sheets to an excel file for analyses


In [29]:
#pip installs
#!pip install python-docx 
#!pip install openpyxl

In [1]:
import docx
import os
from openpyxl import Workbook
import re  # Import the regular expression module
import pandas as pd


In [2]:
path = docx_dir = "C:/Users/anaho/Desktop/research/Ayahuasca/DATA/FinalCodings"
excel_path = "C:/Users/anaho/Desktop/research/Ayahuasca/analysis/codage entretiens/all_codes.xls"

In [3]:
def docx_to_excel_combined(docx_dir):
    """
    Processes all DOCX files in a directory, extracts data from ALL tables,
    and writes the combined, transposed data to a single Excel sheet.
    It processes ALL rows where the first item is like 'A1' from each file.
    The output is transformed to a compact format:
    Participants in rows, with question codes and notes.
    """
    try:
        # 1. Get all DOCX files in the directory, sorted alphanumerically
        docx_files = sorted([f for f in os.listdir(docx_dir) if f.endswith(".docx")][4:5])
        if not docx_files:
            print(f"No DOCX files found in the directory: {docx_dir}")
            return

        workbook = Workbook()
        sheet = workbook.active
        all_data = {}  # Store data from all files, keyed by participant and table
        df = pd.DataFrame()

        # 2. Iterate through each DOCX file
        for docx_file in docx_files:
            docx_path = os.path.join(docx_dir, docx_file)
            try:
                document = docx.Document(docx_path)
                num_tables = len(document.tables)
                print(f"Processing {num_tables} tables in {docx_file}")
                if num_tables == 0:
                    print(f"No tables found in {docx_file}, skipping.")
                    continue

            except docx.exceptions.EmptyPackageError as e:
                print(f"Error reading {docx_file}: {e}.  Skipping this file.")
                continue

            # Extract participant name from filename (remove ".docx")
            participant_name = os.path.splitext(docx_file)[0]  # e.g., "codage_01"

            # Extract participant ID (e.g., "01")
            match = re.search(r"_(\d+)", participant_name)
            if match:
                participant_id = match.group(1)
            else:
                print(f"Could not extract participant ID from filename: {docx_file}. Skipping file.")
                continue
                
            all_table_data= [] #instantiate list
            
            # 3. Iterate through each table in the document
            for table_index, table in enumerate(document.tables):
                print(f"  Processing table {table_index + 1} of {num_tables} in {docx_file}")
                table_data = {}  # Store data for the current table, keyed by question code
                # 4. Extract data from the table, handling LetterNumber rows
                for row in table.rows:
                    cells = [cell.text.strip() for cell in row.cells]  # Remove extra whitespace
                    if cells:
                        first_cell_value = cells[0]
                        if re.match(r"^[A-Za-z]\d+$", first_cell_value):
                            question_code = first_cell_value
                            yes_val = cells[2].lower()
                            no_val = cells[3].lower()
                            note = cells[4]
                            if "x" in yes_val:
                                response = 1
                            elif "x" in no_val:
                                response = 0
                            else:
                                response = "NA"
                            table_data[question_code] = {"response": response, "note": note}
                        else:
                            print(
                                f"    Skipping row in {docx_file}, table {table_index + 1} because the first cell '{first_cell_value}' does not match 'A1' format.")
                    else:
                        print(f"    Skipping empty row in {docx_file}, table {table_index + 1}.")

                print(f"    Extracted data from {docx_file}, table {table_index + 1}: {table_data}")

                # 5. Store data for this participant and question
                all_table_data.append(table_data)  # Use participant_id
                
                #flatten the dic
                all_table_data_flat = {k:v for d in all_table_data for k,v in d.items()}
                
                all_data[(participant_id)] = all_table_data_flat           
            
        
        #print(f"All data: {all_data}")

        # 6. Prepare data for Excel, transposing and combining
        # Prepare data for Excel sheet
        excel_data = []
        # Get all unique questions
        unique_questions = []
        for data in all_data.values():
            for question_code in data.keys():
                if question_code not in unique_questions:
                    unique_questions.append(question_code)

        # Create the header row
        header_row = ["Participants"]  # Changed Header
        for question in unique_questions:
            header_row.extend([f"{question}", f"{question} Note"])
        excel_data.append(header_row)

        # Create participant rows
        participant_ids = sorted(list(set([k[0] for k in all_data.keys()])))  # Get unique participant IDs
        for participant_id in participant_ids:
            participant_row = all_data[participant_id]  # Add participant ID here
            print('ROW', participant_id)
            
            excel_data.append(participant_row)

        # 7. Write to Excel
        #for row in excel_data:
            #sheet.append(row)

        # 8. Save the Excel file
        #workbook.save(excel_path)
        #print(f"Successfully combined data from DOCX files in '{docx_dir}' to '{excel_path}'")

    except Exception as e:
        print(f"An error occurred: {e}")
        
    return excel_data

In [6]:
def docx_to_excel_combined(docx_dir):
    """
    Processes all DOCX files in a directory, extracts data from ALL tables,
    and writes the combined, transposed data to a single Excel sheet.
    It processes ALL rows where the first item is like 'A1' from each file.
    The output is transformed to a compact format:
    Participants in rows, with question codes and notes.
    """
    try:
        # 1. Get all DOCX files in the directory, sorted alphanumerically
        docx_files = sorted([f for f in os.listdir(docx_dir) if f.endswith(".docx")])
        if not docx_files:
            print(f"No DOCX files found in the directory: {docx_dir}")
            return

        workbook = Workbook()
        sheet = workbook.active
        all_data = {}  # Store data from all files, keyed by participant ID
        df = pd.DataFrame()

        # 2. Iterate through each DOCX file
        for docx_file in docx_files:
            docx_path = os.path.join(docx_dir, docx_file)
            try:
                document = docx.Document(docx_path)
                num_tables = len(document.tables)
                print(f"Processing {num_tables} tables in {docx_file}")
                if num_tables == 0:
                    print(f"No tables found in {docx_file}, skipping.")
                    continue

            except docx.exceptions.EmptyPackageError as e:
                print(f"Error reading {docx_file}: {e}.  Skipping this file.")
                continue

            # Extract participant name from filename (remove ".docx")
            participant_name = os.path.splitext(docx_file)[0]  # e.g., "codage_01"
            

            # Extract participant ID (e.g., "01")
            participant_id = participant_name[-2:]

            all_table_data= [] #instantiate list

            # 3. Iterate through each table in the document
            for table_index, table in enumerate(document.tables):
                print(f"  Processing table {table_index + 1} of {num_tables} in {docx_file}")
                table_data = {}  # Store data for the current table, keyed by question code
                # 4. Extract data from the table, handling LetterNumber rows
                for row in table.rows:
                    cells = [cell.text.strip() for cell in row.cells]  # Remove extra whitespace
                    if cells:
                        first_cell_value = cells[0]
                        if re.match(r"^[A-Za-z]\d+$", first_cell_value): #accounting for the questions of the form A1
                            question_code = first_cell_value
                            yes_val = cells[2].lower()
                            no_val = cells[3].lower()
                            note = cells[4]
                            if "x" in yes_val:
                                response = 1
                            elif "x" in no_val:
                                response = 0
                            else:
                                response = "NA"
                            table_data[question_code] = {"response": response, "note": note}
                        elif re.match(r"^[A-Za-z]\d[A-Za-z]+$", first_cell_value): #accouting for the questions of the form A1b
                            question_code = first_cell_value
                            yes_val = cells[2].lower()
                            no_val = cells[3].lower()
                            note = cells[4]
                            if "x" in yes_val:
                                response = 1
                            elif "x" in no_val:
                                response = 0
                            else:
                                response = "NA"
                            table_data[question_code] = {"response": response, "note": note}
                        else:
                            print(
                                f"    Skipping row in {docx_file}, table {table_index + 1} because the first cell '{first_cell_value}' does not match 'A1' format.")
                    else:
                        print(f"    Skipping empty row in {docx_file}, table {table_index + 1}.")

                print(f"    Extracted data from {docx_file}, table {table_index + 1}: {table_data}")

                # 5. Store data for this participant and question
                all_table_data.append(table_data)  # Use participant_id

            # Flatten the list of dictionaries into a single dictionary for the participant
            all_table_data_flat = {}
            for item in all_table_data:
                all_table_data_flat.update(item)

            all_data[participant_id] = all_table_data_flat


        #print(f"All data: {all_data}")

        # 6. Prepare data for Excel, transposing and combining
        # Prepare data for Excel sheet
        excel_data = []
        # Get all unique questions
        unique_questions = set()
        for data in all_data.values():
            unique_questions.update(data.keys())
        unique_questions = sorted(list(unique_questions))

        # Create the header row
        header_row = ["Participants"]  # Changed Header
        for question in unique_questions:
            header_row.extend([f"{question}", f"{question} Note"])
        excel_data.append(header_row)

        # Create participant rows
        participant_ids = sorted(list(all_data.keys()))  # Get unique participant IDs
        for participant_id in participant_ids:
            participant_data = all_data[participant_id]
            participant_row = [participant_id]  # Add participant ID here
            for question in unique_questions:
                if question in participant_data:
                    participant_row.extend([participant_data[question]['response'], participant_data[question]['note']])
                else:
                    participant_row.extend(["", ""])  # Add empty cells if question not answered
            excel_data.append(participant_row)

        # 7. Write to Excel
        for row in excel_data:
            sheet.append(row)

        # 8. Save the Excel file
        excel_path = os.path.join(docx_dir, "combined_data.xlsx")
        workbook.save(excel_path)
        print(f"Successfully combined data from DOCX files in '{docx_dir}' to '{excel_path}'")

    except Exception as e:
        print(f"An error occurred: {e}")

    return excel_data


In [7]:
sheet = docx_to_excel_combined(path)

Processing 3 tables in New_Coding_01.docx
  Processing table 1 of 3 in New_Coding_01.docx
    Skipping row in New_Coding_01.docx, table 1 because the first cell 'Participant code: Sally-01' does not match 'A1' format.
    Skipping row in New_Coding_01.docx, table 1 because the first cell 'A' does not match 'A1' format.
    Extracted data from New_Coding_01.docx, table 1: {'A1': {'response': 0, 'note': ''}, 'A2': {'response': 0, 'note': ''}, 'A3': {'response': 0, 'note': ''}, 'A4': {'response': 0, 'note': ''}, 'A5': {'response': 0, 'note': ''}, 'A6': {'response': 0, 'note': '.'}, 'A7': {'response': 0, 'note': ''}, 'A8': {'response': 0, 'note': ''}, 'A9': {'response': 0, 'note': ''}}
  Processing table 2 of 3 in New_Coding_01.docx
    Skipping row in New_Coding_01.docx, table 2 because the first cell 'B' does not match 'A1' format.
    Skipping row in New_Coding_01.docx, table 2 because the first cell 'C' does not match 'A1' format.
    Skipping row in New_Coding_01.docx, table 2 because